# Amazon MemoryDB

>[Vector Search](https://docs.aws.amazon.com/memorydb/latest/devguide/vector-search.html/) 入门和 langchain 集成指南。

## 什么是 Amazon MemoryDB？

MemoryDB 与流行的开源数据存储 Redis OSS 兼容，使您能够利用其灵活且用户友好的 Redis OSS 数据结构、API 和命令快速构建应用程序。借助 MemoryDB，所有数据都存储在内存中，从而实现微秒级的读取延迟和个位数毫秒级的写入延迟以及高吞吐量。MemoryDB 还使用多可用区 (AZ) 事务日志将数据持久存储在多个可用区 (AZ) 中，从而实现快速故障转移、数据库恢复和节点重启。


## MemoryDB 的向量搜索

MemoryDB 的向量搜索扩展了 MemoryDB 的功能。向量搜索可与现有的 MemoryDB 功能结合使用。不使用向量搜索的应用程序不受其影响。MemoryDB 在所有可用区域都提供向量搜索。您可以使用现有的 MemoryDB 数据或 Redis OSS API 来构建机器学习和生成式 AI 用例，例如检索增强生成、异常检测、文档检索和实时推荐。

* 对 Redis hashes 和 `JSON` 中的多个字段进行索引
* 向量相似性搜索（支持 `HNSW` (ANN) 或 `FLAT` (KNN)）
* 向量范围搜索（例如，查找查询向量半径范围内的所有向量）
* 增量索引，无性能损失

## 设置


### 安装 Redis Python 客户端

`Redis-py` 是一个可用于连接 MemoryDB 的 Python 客户端。

In [ ]:
%pip install --upgrade --quiet  redis langchain-aws

In [3]:
from langchain_aws.embeddings import BedrockEmbeddings

embeddings = BedrockEmbeddings()

### MemoryDB 连接

有效的 Redis URL 模式包括：
1. `redis://`  - 连接到 Redis 集群，未加密
2. `rediss://` - 连接到 Redis 集群，已启用 TLS 加密

有关其他连接参数的更多信息，请参阅 [redis-py 文档](https://redis-py.readthedocs.io/en/stable/connections.html)。

### 示例数据

首先，我们将描述一些示例数据，以便演示 Redis 向量存储的各种属性。

In [4]:
metadata = [
    {
        "user": "john",
        "age": 18,
        "job": "engineer",
        "credit_score": "high",
    },
    {
        "user": "derrick",
        "age": 45,
        "job": "doctor",
        "credit_score": "low",
    },
    {
        "user": "nancy",
        "age": 94,
        "job": "doctor",
        "credit_score": "high",
    },
    {
        "user": "tyler",
        "age": 100,
        "job": "engineer",
        "credit_score": "high",
    },
    {
        "user": "joe",
        "age": 35,
        "job": "dentist",
        "credit_score": "medium",
    },
]
texts = ["foo", "foo", "foo", "bar", "bar"]
index_name = "users"

### 创建 MemoryDB 向量存储

InMemoryVectorStore 实例可以使用以下方法进行初始化
- ``InMemoryVectorStore.__init__`` - 直接初始化
- ``InMemoryVectorStore.from_documents`` - 从 ``Langchain.docstore.Document`` 对象列表初始化
- ``InMemoryVectorStore.from_texts`` - 从文本列表（可选带元数据）初始化
- ``InMemoryVectorStore.from_existing_index`` - 从现有的 MemoryDB 索引初始化

In [7]:
from langchain_aws.vectorstores.inmemorydb import InMemoryVectorStore

vds = InMemoryVectorStore.from_texts(
    embeddings,
    redis_url="rediss://cluster_endpoint:6379/ssl=True ssl_cert_reqs=none",
)

In [6]:
vds.index_name

'users'

## 查询

根据您的具体用例，有多种方法可以查询 ``InMemoryVectorStore`` 实现：

- ``similarity_search``：查找给定向量最相似的向量。
- ``similarity_search_with_score``：查找给定向量最相似的向量并返回向量距离。
- ``similarity_search_limit_score``：查找给定向量最相似的向量，并将结果数量限制为 ``score_threshold``。
- ``similarity_search_with_relevance_scores``：查找给定向量最相似的向量并返回向量相似度。
- ``max_marginal_relevance_search``：查找给定向量最相似的向量，同时优化多样性。

In [10]:
results = vds.similarity_search("foo")
print(results[0].page_content)

foo


In [12]:
# with scores (distances)
results = vds.similarity_search_with_score("foo", k=5)
for result in results:
    print(f"Content: {result[0].page_content} --- Score: {result[1]}")

Content: foo --- Score: 0.0
Content: foo --- Score: 0.0
Content: foo --- Score: 0.0
Content: bar --- Score: 0.1566
Content: bar --- Score: 0.1566


In [13]:
# limit the vector distance that can be returned
results = vds.similarity_search_with_score("foo", k=5, distance_threshold=0.1)
for result in results:
    print(f"Content: {result[0].page_content} --- Score: {result[1]}")

Content: foo --- Score: 0.0
Content: foo --- Score: 0.0
Content: foo --- Score: 0.0


In [14]:
# with scores
results = vds.similarity_search_with_relevance_scores("foo", k=5)
for result in results:
    print(f"Content: {result[0].page_content} --- Similiarity: {result[1]}")

Content: foo --- Similiarity: 1.0
Content: foo --- Similiarity: 1.0
Content: foo --- Similiarity: 1.0
Content: bar --- Similiarity: 0.8434
Content: bar --- Similiarity: 0.8434


In [16]:
# you can also add new documents as follows
new_document = ["baz"]
new_metadata = [{"user": "sam", "age": 50, "job": "janitor", "credit_score": "high"}]
# both the document and metadata must be lists
vds.add_texts(new_document, new_metadata)

['doc:users:b9c71d62a0a34241a37950b448dafd38']

## 将 MemoryDB 用作检索器

在这里，我们将介绍使用向量存储作为检索器的不同选项。

我们可以使用三种不同的搜索方法进行检索。默认情况下，它将使用语义相似度。

In [26]:
query = "foo"
results = vds.similarity_search_with_score(query, k=3, return_metadata=True)

for result in results:
    print("Content:", result[0].page_content, " --- Score: ", result[1])

Content: foo  --- Score:  0.0
Content: foo  --- Score:  0.0
Content: foo  --- Score:  0.0


In [27]:
retriever = vds.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [28]:
docs = retriever.invoke(query)
docs

[Document(page_content='foo', metadata={'id': 'doc:users_modified:988ecca7574048e396756efc0e79aeca', 'user': 'john', 'job': 'engineer', 'credit_score': 'high', 'age': '18'}),
 Document(page_content='foo', metadata={'id': 'doc:users_modified:009b1afeb4084cc6bdef858c7a99b48e', 'user': 'derrick', 'job': 'doctor', 'credit_score': 'low', 'age': '45'}),
 Document(page_content='foo', metadata={'id': 'doc:users_modified:7087cee9be5b4eca93c30fbdd09a2731', 'user': 'nancy', 'job': 'doctor', 'credit_score': 'high', 'age': '94'}),
 Document(page_content='bar', metadata={'id': 'doc:users_modified:01ef6caac12b42c28ad870aefe574253', 'user': 'tyler', 'job': 'engineer', 'credit_score': 'high', 'age': '100'})]

还有一个 `similarity_distance_threshold` 检索器，它允许用户指定向量距离

In [29]:
retriever = vds.as_retriever(
    search_type="similarity_distance_threshold",
    search_kwargs={"k": 4, "distance_threshold": 0.1},
)

In [30]:
docs = retriever.invoke(query)
docs

[Document(page_content='foo', metadata={'id': 'doc:users_modified:988ecca7574048e396756efc0e79aeca', 'user': 'john', 'job': 'engineer', 'credit_score': 'high', 'age': '18'}),
 Document(page_content='foo', metadata={'id': 'doc:users_modified:009b1afeb4084cc6bdef858c7a99b48e', 'user': 'derrick', 'job': 'doctor', 'credit_score': 'low', 'age': '45'}),
 Document(page_content='foo', metadata={'id': 'doc:users_modified:7087cee9be5b4eca93c30fbdd09a2731', 'user': 'nancy', 'job': 'doctor', 'credit_score': 'high', 'age': '94'})]

最后，`similarity_score_threshold` 允许用户定义相似文档的最低分数。

In [31]:
retriever = vds.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.9, "k": 10},
)

In [32]:
retriever.invoke("foo")

[Document(page_content='foo', metadata={'id': 'doc:users_modified:988ecca7574048e396756efc0e79aeca', 'user': 'john', 'job': 'engineer', 'credit_score': 'high', 'age': '18'}),
 Document(page_content='foo', metadata={'id': 'doc:users_modified:009b1afeb4084cc6bdef858c7a99b48e', 'user': 'derrick', 'job': 'doctor', 'credit_score': 'low', 'age': '45'}),
 Document(page_content='foo', metadata={'id': 'doc:users_modified:7087cee9be5b4eca93c30fbdd09a2731', 'user': 'nancy', 'job': 'doctor', 'credit_score': 'high', 'age': '94'})]

In [13]:
retriever.invoke("foo")

[Document(page_content='foo', metadata={'id': 'doc:users:8f6b673b390647809d510112cde01a27', 'user': 'john', 'job': 'engineer', 'credit_score': 'high', 'age': '18'}),
 Document(page_content='bar', metadata={'id': 'doc:users:93521560735d42328b48c9c6f6418d6a', 'user': 'tyler', 'job': 'engineer', 'credit_score': 'high', 'age': '100'}),
 Document(page_content='foo', metadata={'id': 'doc:users:125ecd39d07845eabf1a699d44134a5b', 'user': 'nancy', 'job': 'doctor', 'credit_score': 'high', 'age': '94'}),
 Document(page_content='foo', metadata={'id': 'doc:users:d6200ab3764c466082fde3eaab972a2a', 'user': 'derrick', 'job': 'doctor', 'credit_score': 'low', 'age': '45'})]

## 删除索引

要删除条目，您必须按其键来寻址。

In [34]:
# delete the indices too
InMemoryVectorStore.drop_index(
    index_name="users", delete_documents=True, redis_url="redis://localhost:6379"
)
InMemoryVectorStore.drop_index(
    index_name="users_modified",
    delete_documents=True,
    redis_url="redis://localhost:6379",
)

True